<a href="https://colab.research.google.com/github/byambaa1982/mongol_tweets/blob/master/tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis

### Aшигласан сангууд

In [9]:
#Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import tweepy
import pandas as pd
import numpy as np
import regex as re
import os
import json
import time
import sys
import io
from datetime import datetime, date, time, timedelta
import pandas_gbq


# Google Cloud Platform - Using Machine Learning APIs  ).

### Security

First things first - we need to authenticate against the Google Cloud APIs.

#### Getting a Google API Credential.

First, visit <a href="http://console.cloud.google.com/apis">API console</a>, choose "Credentials" on the left-hand menu.  Choose "Create Credentials" and generate an API key for your application. You should probably restrict it by IP address to prevent abuse, but for now, just  leave that field blank and delete the API key after trying out this demo.

Then, when you have your key, you will enter it in this first executable cell:

In [20]:
import getpass

APIKEY = getpass.getpass()

··········


 ### Access google driver using google credentials

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [22]:
os.chdir('/content/gdrive/My Drive/fiverr/tweets')
with open("twitter_credentials.json", "r") as file:
  creds = json.load(file)
auth = tweepy.OAuthHandler(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])
auth.set_access_token(creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
api = tweepy.API(auth, wait_on_rate_limit=True)
os.chdir('/content/gdrive/My Drive/fiverr/byamba')

### Pull data using keyword and date

In [23]:
def get_tweets_csv(search_words, date_since):
  new_search = search_words + " -filter:retweets"

  tweets = tweepy.Cursor(api.search,
                              tweet_mode="extended", 
                              q=new_search,
                              lang="ru",
                              since=date_since).items(2000)

  users_locs = [[tweet.id_str, tweet.user.screen_name, tweet.full_text, tweet.created_at, tweet.user.location
                ,tweet.favorite_count,tweet.retweet_count] for tweet in tweets]
  users_locs
  # Checks if there are coordinates attached to tweets, if so extracts them
  # users_locs['Tweet Coordinates'] = users_locs.apply(extract_coordinates,axis=1)

  tweet_text = pd.DataFrame(data=users_locs, 
                      columns=["id", "user", "tweets","date","location","favorites","retweets"])

  # tweet_text['Tweet Coordinates'] = tweet_text.apply(extract_coordinates,axis=1)
  # tweet_text.to_csv(search_words+".csv", index=False)
  return tweet_text

### Sort df by favorites

In [25]:
df=get_tweets_csv("Хүрэлсүх", "2021-05-12")
df=df.sort_values("favorites", ascending=False)
df.head(6)
df.to_csv('khurelsukh.csv',index=False)
df1=pd.read_csv('khurelsukh.csv')
df1=df1[["user","tweets","favorites", "date"]]
df1.head(6)
df2=df1.head(100)
df2

,user,tweets,favorites,date
0,DanzanBolormaa,За Энхбат Хүрэлсүх 2ын ярилцлага үзлээ. Боловс...,274,2021-05-19 04:41:43
1,g_ganbat,АНУ-ын Мянганы сорилын сангийн УБ-ын усан ханг...,242,2021-05-18 11:37:34
2,Mandolinchin,"Нэр дэвшиж буй 3 эрхэм Энхбат ах, найз Хүрэлсү...",183,2021-05-20 07:39:09
3,tvdaria,С.Эрдэнэ 2007онд Солонгост ажиллаж бсан иргэди...,162,2021-05-17 05:25:07
4,batengineer,"Хүрэлсүх, Эрдэнэ нарын сурталчилгаа интернет о...",155,2021-05-17 02:41:41
...,...,...,...,...
95,amarsanaa,У.Хүрэлсүх: Манай Засгийн газар өрийн удирдлаг...,10,2021-05-19 03:38:10
96,Burmaa87,МАНай Хүрэлсүх би бол эгэл ардын төлөөлөл гэсэ...,10,2021-05-19 00:20:56
97,sakuna_mn,У.Хүрэлсүх: Аав ээжийгээ амьд сэрүүн байхад нь...,10,2021-05-20 14:36:07
98,ariunq,"Хүрэлсүх аав ээжийнхээ ачийг хариулсан, цуйван...",10,2021-05-21 00:12:11


In [31]:
print('Total tweets:{}, Total favorites:{}'.format(df.shape[0], df.favorites.sum()))

Total tweets:803, Total favorites:5424


### Sentiment analysis using Google NLP API

In [26]:
lservice = build('language', 'v1beta1', developerKey=APIKEY)
def do_sentiment(quote):
  response = lservice.documents().analyzeSentiment(
  body={
    'document': {
        'type': 'PLAIN_TEXT',
        'content': quote
    }
  }).execute()
  polarity = response['documentSentiment']['polarity']
  # magnitude = response['documentSentiment']['magnitude']
  return polarity

### Before calling NLP API, we need to translate Mongolian tweets to English
### using google translate API

In [24]:
# running Translate API
from googleapiclient.discovery import build
service = build('translate', 'v2', developerKey=APIKEY)

# use the service
inputs = ['is it really this easy?', 'amazing technology', 'wow']
outputs = service.translations().list(source='en', target='mn', q=inputs).execute()
# print outputs
for input, output in zip(inputs, outputs['translations']):
  print(u"{0} -> {1}".format(input, output['translatedText']))

is it really this easy? -> энэ үнэхээр амархан уу?
amazing technology -> гайхалтай технологи
wow -> хөөх


In [4]:
def translate_it(inputs):
  outputs = service.translations().list(source='mn', target='en', q=inputs).execute()
  return outputs['translations'][0]['translatedText']

### Run NLP API over each tweets to find polarity

In [28]:
df2['english']=df2.tweets.map(lambda x:translate_it(x))
df2['polarity']=df2.english.map(lambda x:do_sentiment(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [30]:
df2=df2[['user','polarity','tweets','favorites']]
df2

,user,polarity,tweets,favorites
0,DanzanBolormaa,0.4,За Энхбат Хүрэлсүх 2ын ярилцлага үзлээ. Боловс...,274
1,g_ganbat,-0.8,АНУ-ын Мянганы сорилын сангийн УБ-ын усан ханг...,242
2,Mandolinchin,0.9,"Нэр дэвшиж буй 3 эрхэм Энхбат ах, найз Хүрэлсү...",183
3,tvdaria,-0.1,С.Эрдэнэ 2007онд Солонгост ажиллаж бсан иргэди...,162
4,batengineer,-1.0,"Хүрэлсүх, Эрдэнэ нарын сурталчилгаа интернет о...",155
...,...,...,...,...
95,amarsanaa,-0.1,У.Хүрэлсүх: Манай Засгийн газар өрийн удирдлаг...,10
96,Burmaa87,-1.0,МАНай Хүрэлсүх би бол эгэл ардын төлөөлөл гэсэ...,10
97,sakuna_mn,1.0,У.Хүрэлсүх: Аав ээжийгээ амьд сэрүүн байхад нь...,10
98,ariunq,-0.4,"Хүрэлсүх аав ээжийнхээ ачийг хариулсан, цуйван...",10


In [17]:
print('Total tweets:{}, Total favorites:{}'.format(df.shape[0], df.favorites.sum()))

Total tweets:951, Total favorites:6243


In [15]:
# table_id='tweets.enkhbat'
# project_id = 'twittersheet-275317'
# pandas_gbq.to_gbq(df, table_id, project_id=project_id ,if_exists='replace')

1it [00:02,  2.77s/it]
